In [1]:
!pip install boto3

In [2]:
# Cell 1 — Imports + local paths

import os
from pathlib import Path
import boto3
from botocore.exceptions import ClientError

LOCAL_DATA_DIR = Path(r"C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data")

print("Local data directory set to:", LOCAL_DATA_DIR)
print("Exists?", LOCAL_DATA_DIR.exists())
print("Is directory?", LOCAL_DATA_DIR.is_dir())


Local data directory set to: C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data
Exists? True
Is directory? True


In [4]:
# Cell 2A — Put your provided keys into environment variables (runtime only)

import os

os.environ["RUNPOD_S3_ACCESS_KEY"] = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
os.environ["RUNPOD_S3_SECRET_KEY"] = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"

print("✅ Environment variables set for this notebook session.")
print("Access present?", "RUNPOD_S3_ACCESS_KEY" in os.environ)
print("Secret present?", "RUNPOD_S3_SECRET_KEY" in os.environ)


✅ Environment variables set for this notebook session.
Access present? True
Secret present? True


In [6]:
# Cell 2 — RunPod S3 credentials + destination settings (using given access details)

ACCESS_KEY = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
SECRET_KEY = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"

BUCKET = "e9tcw5eupu"
ENDPOINT_URL = "https://s3api-eu-ro-1.runpod.io"
REGION = "eu-ro-1"

DEST_ROOT = "tight loose clothing model new"
DEST_DATA_PREFIX = f"{DEST_ROOT}/data"

print("✅ Credentials and settings loaded.")
print("Bucket:", BUCKET)
print("Endpoint:", ENDPOINT_URL)
print("Region:", REGION)
print("Destination prefix:", DEST_DATA_PREFIX)
print("Access key set?", bool(ACCESS_KEY))
print("Secret key set?", bool(SECRET_KEY))


✅ Credentials and settings loaded.
Bucket: e9tcw5eupu
Endpoint: https://s3api-eu-ro-1.runpod.io
Region: eu-ro-1
Destination prefix: tight loose clothing model new/data
Access key set? True
Secret key set? True


In [7]:
# Cell 3 — Create S3 client + connectivity check

s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
)

print("✅ S3 client created. Checking access to bucket...")

try:
    resp = s3.list_objects_v2(Bucket=BUCKET, MaxKeys=5)
    print("✅ Connected successfully.")
    print("KeyCount returned:", resp.get("KeyCount", 0))
except ClientError as e:
    print("❌ Connection / permission error while accessing the bucket.")
    print("Error:", e)
    raise


✅ S3 client created. Checking access to bucket...
✅ Connected successfully.
KeyCount returned: 5


In [8]:
# Cell 4 — Scan local folder (recursive)

if not LOCAL_DATA_DIR.exists() or not LOCAL_DATA_DIR.is_dir():
    raise FileNotFoundError(f"❌ Local folder not found: {LOCAL_DATA_DIR}")

all_files = [p for p in LOCAL_DATA_DIR.rglob("*") if p.is_file()]

total_files = len(all_files)
total_bytes = sum(p.stat().st_size for p in all_files)

print("✅ Local folder scanned.")
print(f"Total files found: {total_files}")
print(f"Total size (bytes): {total_bytes:,}")

print("\nSample files:")
for p in all_files[:10]:
    print(" -", p)


✅ Local folder scanned.
Total files found: 240
Total size (bytes): 40,407,193

Sample files:
 - C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data\upper body clothing categories\blouse\Screenshot 2026-01-23 190627.png
 - C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data\upper body clothing categories\blouse\Screenshot 2026-01-23 190652.png
 - C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data\upper body clothing categories\blouse\Screenshot 2026-01-23 190819.png
 - C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data\upper body clothing categories\blouse\Screenshot 2026-01-23 190837.png
 - C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data\upper body clothing categories\blouse\Screenshot 2026-01-23 190859.png
 - C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data\

In [ ]:
# Cell 5 — Upload all files to RunPod S3: tight loose clothing model new/data/<relative_path>

def upload_directory(files, local_base: Path, bucket: str, dest_prefix: str):
    uploaded = 0
    failed = 0
    bytes_uploaded = 0

    print("\n🚀 Starting upload...")
    print("Local base:", local_base)
    print("Bucket:", bucket)
    print("Destination prefix:", dest_prefix)
    print("-" * 70)

    for idx, file_path in enumerate(files, start=1):
        rel_path = file_path.relative_to(local_base).as_posix()
        s3_key = f"{dest_prefix}/{rel_path}"

        try:
            print(f"[{idx}/{len(files)}] Uploading: {rel_path}")
            s3.upload_file(str(file_path), bucket, s3_key)

            uploaded += 1
            bytes_uploaded += file_path.stat().st_size

            if uploaded % 50 == 0 or idx == len(files):
                print(f"   ✅ Progress -> uploaded={uploaded}, failed={failed}, bytes_uploaded={bytes_uploaded:,}")

        except Exception as e:
            failed += 1
            print(f"   ❌ FAILED: {rel_path}")
            print("   Error:", repr(e))

    print("\n✅ Upload finished.")
    print(f"Uploaded: {uploaded}")
    print(f"Failed:   {failed}")
    print(f"Bytes uploaded: {bytes_uploaded:,}")

    return {"uploaded": uploaded, "failed": failed, "bytes_uploaded": bytes_uploaded}


results = upload_directory(
    files=all_files,
    local_base=LOCAL_DATA_DIR,
    bucket=BUCKET,
    dest_prefix=DEST_DATA_PREFIX
)

print("\nRESULTS:", results)



🚀 Starting upload...
Local base: C:\Users\HP\Desktop\Jupyter Notebooks\Primed\Loose and Tight clothes classification\data
Bucket: e9tcw5eupu
Destination prefix: tight loose clothing model new/data
----------------------------------------------------------------------
[1/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190627.png
[2/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190652.png
[3/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190819.png
[4/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190837.png
[5/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190859.png
[6/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190921.png
[7/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 190941.png
[8/240] Uploading: upper body clothing categories/blouse/Screenshot 2026-01-23 191004.png
[9/240] Upl

In [ ]:
# Cell 6 — Verify upload by listing some uploaded objects

prefix = f"{DEST_DATA_PREFIX}/"
print("🔎 Verifying upload. Listing objects under:", prefix)

resp = s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix, MaxKeys=50)
contents = resp.get("Contents", [])

print("Objects found:", len(contents))
for obj in contents[:20]:
    print(" -", obj["Key"], "|", obj["Size"], "bytes")

if resp.get("IsTruncated"):
    print("⚠️ Listing truncated (more than 50 objects exist).")
